# Milestone 2 project: Political Pope

Neccesary imports

In [5]:
import pandas as pd
import bz2
import json
import re
import math
import pickle
import statistics as stat
import matplotlib.pyplot as plt
import numpy as np
import time
import os
import string
from langdetect import detect # for quotes language filtering
import seaborn as sns # for data visualisation
import fastcluster

# NLP libraries
import nltk, spacy, sklearn
from spacy.lang.en import English

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize # useful ?
from nltk.tokenize import TweetTokenizer

import vaderSentiment
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

# PCA steps
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/lucamusumeci/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/lucamusumeci/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/lucamusumeci/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [6]:
lemmatizer = nltk.stem.WordNetLemmatizer()
tokenizer = TweetTokenizer()

In [7]:
Path = "../data"
df = pd.DataFrame()
for entry in os.scandir(Path):
    if (entry.path.endswith(".pkl") and entry.is_file()):
        file = open(entry,'rb')
        data = pickle.load(file)
        df = df.append(data)

In [17]:
df

,quote,speaker,probability,second_speaker,second_probability,third_speaker,third_probability,date,nbr_occurences,phase,nbr_words,nbr_characters,nbr_speakers
0,"' It is not now, nor has it ever been, the gol...",Bernie Sanders,0.5395,None,0.3128,Hillary Clinton,0.1477,2015-10-25 14:12:35,1,E,73,359,3
1,But if you talk about why the middle class is ...,Bernie Sanders,0.6011,None,0.3673,Hillary Clinton,0.0141,2015-10-15 10:07:01,1,E,44,227,6
2,"If they destroy you, who is there left?",Bernie Sanders,0.9061,None,0.0939,NaN,NaN,2015-04-15 17:49:00,1,E,8,39,2
3,It would make everybody in America poorer.,Bernie Sanders,0.5619,None,0.277,Ezra Klein,0.0821,2015-07-30 01:49:15,13,E,7,42,5
4,"labor is the source of all wealth,",Bernie Sanders,0.806,None,0.1309,Karl Marx,0.0304,2015-10-14 20:09:23,1,E,7,34,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...
102988,the Gospel of Jesus Christ is the greatest jus...,Pope Francis,0.8534,None,0.1466,NaN,NaN,2020-03-11 15:05:20,1,E,33,165,2
102989,"from the bishop to the foreigner, from the pri...",Pope Francis,0.7182,None,0.2818,NaN,NaN,2020-01-17 16:07:41,1,E,46,278,2
102990,"`saw the son coming from afar,' that son who h...",Pope Francis,0.7372,None,0.2628,NaN,NaN,2020-03-20 13:13:06,1,E,20,96,2
102991,This prayer of the Angelus today is a little s...,Pope Francis,0.7234,None,0.2766,NaN,NaN,2020-03-08 20:57:39,1,E,28,125,2


## Pre-processing

The first step will be to change the probabilities (type = object) into float numbers to evaluate them properly later on.

In [23]:
new_df = df.copy()

#modify the types of the probabilities (from Object to float)
new_df['probability'] = new_df['probability'].astype(np.float)
new_df['second_probability'] = new_df['second_probability'].astype(np.float)
new_df['third_probability'] = new_df['third_probability'].astype(np.float)
new_df

,quote,speaker,probability,second_speaker,second_probability,third_speaker,third_probability,date,nbr_occurences,phase,nbr_words,nbr_characters,nbr_speakers
0,"' It is not now, nor has it ever been, the gol...",Bernie Sanders,0.5395,None,0.3128,Hillary Clinton,0.1477,2015-10-25 14:12:35,1,E,73,359,3
1,But if you talk about why the middle class is ...,Bernie Sanders,0.6011,None,0.3673,Hillary Clinton,0.0141,2015-10-15 10:07:01,1,E,44,227,6
2,"If they destroy you, who is there left?",Bernie Sanders,0.9061,None,0.0939,NaN,NaN,2015-04-15 17:49:00,1,E,8,39,2
3,It would make everybody in America poorer.,Bernie Sanders,0.5619,None,0.2770,Ezra Klein,0.0821,2015-07-30 01:49:15,13,E,7,42,5
4,"labor is the source of all wealth,",Bernie Sanders,0.8060,None,0.1309,Karl Marx,0.0304,2015-10-14 20:09:23,1,E,7,34,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...
102988,the Gospel of Jesus Christ is the greatest jus...,Pope Francis,0.8534,None,0.1466,NaN,NaN,2020-03-11 15:05:20,1,E,33,165,2
102989,"from the bishop to the foreigner, from the pri...",Pope Francis,0.7182,None,0.2818,NaN,NaN,2020-01-17 16:07:41,1,E,46,278,2
102990,"`saw the son coming from afar,' that son who h...",Pope Francis,0.7372,None,0.2628,NaN,NaN,2020-03-20 13:13:06,1,E,20,96,2
102991,This prayer of the Angelus today is a little s...,Pope Francis,0.7234,None,0.2766,NaN,NaN,2020-03-08 20:57:39,1,E,28,125,2


### A) Speaker probability thresholding

Each quote has a probability assigned by the algorithm that indicates the certainity (or not) it was said from the given speaker. One possible way is to put a threshold of 50% below which a quote will be removed fropm dataset. Here is the logic behind a 50% threshold: if probability >= 0.5, it means we are more confident that the speaker truly said the quote than someone else said it.

In [ ]:
new_df.drop(new_df.query('probability <= 0.5').index, inplace=True)
new_df = new_df.reset_index(drop=True)

### B) Non-english quotes removal

Quotebank dataset is composed of quotes of different languages. In fact the machinery(find better term) craved on websites of different languages. To perform good NLP and for the sake of comparing what is comparable, we'll remove every quote that isn't considered as english. This task will be done with the NLP language detection library 'langdetect' that supports over 55 different languages.

detect_langs("Otec matka syn.")
[sk:0.572770823327, pl:0.292872522702, cs:0.134356653968]
could be used to solidy quotes selection with p(english) >= 0.8 for example

In [ ]:
index = []

for n, quote_ in enumerate(new_df['quote']) :
    value = detect(quote_)
    if value != "en":
        index.append(n)
        
new_df.drop(index, inplace=True)
new_df = new_df.reset_index(drop=True)

print("Percentage of non-english quotes that were removed :", round((len(df)-len(new_df))/len(df)*100, 2))

### C) NLP basic pre-processing steps

To handle text properly for natural language processing, a set of classical operations must be done:

In [19]:
#1 remove digits
new_df['pre_processed_quote'] = new_df['quote'].astype(str).str.replace('\d+', '', regex=True)

NameError: name 'new_df' is not defined

In [20]:
#2 remove punctuation
new_df['pre_processed_quote'] = new_df['pre_processed_quote'].apply(remove_punctuation)

NameError: name 'new_df' is not defined

In [21]:
#3 remove spaces at the beginning and ending of quotes
new_df['pre_processed_quote'] = new_df['pre_processed_quote'].str.strip()

NameError: name 'new_df' is not defined

In [22]:
#4 remove rows with empty or NaN quotes (so also the ones which only had digits &/or punctuation)
new_df['pre_processed_quote'].replace("", np.nan, inplace=True)
new_df.dropna(subset = ["pre_processed_quote"], inplace=True)

NameError: name 'new_df' is not defined

In [ ]:
#5 lower the letters of the words
new_df['pre_processed_quote'] = new_df['pre_processed_quote'].str.lower()

In [24]:
#6 Quotes tokenization (useful for the removal of stop words)
new_df['tokenized_quote'] = new_df['quote'].apply(tokenization) #important for features (voc)

IndentationError: unexpected indent (<ipython-input-24-6603d052bcee>, line 2)

In [25]:
#7 Quotes lemmatization to simplify the NLP. Helps to focus on the meaning of the word instead of its variations.
new_df['lemmatized_quote'] = new_df['tokenized_quote'].apply(lemmatization)

KeyError: 'tokenized_quote'

In [27]:
#8 remove stop words
stop_words = stopwords.words('english')
stop_words.extend(spacy.lang.en.stop_words.STOP_WORDS)
stop_words.extend([singer.capitalize() for singer in stop_words]) #stop words with capitale
stop_words.extend(list(string.punctuation))

new_df['pre_processed_quote'] = new_df.apply(lambda x: remove_stop_words(x['tokenized_quote'], stop_words), axis=1)

NameError: name 'remove_stop_words' is not defined

## Exploratory analysis

## Features extraction (data processing)

## PCA